# 기대 SARSA 이해하기: 완전 가이드

# 목차

- [소개](#introduction)
- [기대 SARSA란?](#what-is-expected-sarsa)
- [기대 SARSA는 어디서 어떻게 사용되는가](#where-and-how-expected-sarsa-is-used)
- [기대 SARSA의 수학적 기초](#mathematical-foundation-of-expected-sarsa)
  - [복잡한 원본 버전](#complex-original-version)
  - [간단한 버전](#simplified-version)
- [기대 SARSA 단계별 설명](#step-by-step-explanation-of-expected-sarsa)
- [기대 SARSA의 핵심 요소](#key-components-of-expected-sarsa)
  - [Q-테이블](#q-table)
  - [탐험 대 활용 (정책)](#exploration-vs-exploitation-policy)
  - [학습률 (α)](#learning-rate-a)
  - [할인 인자 (γ)](#discount-factor-g)
- [기대 SARSA vs. SARSA vs. Q-러닝](#expected-sarsa-vs-sarsa-vs-q-learning)
- [실용 예제: 그리드 월드](#practical-example-grid-world)
- [환경 설정](#setting-up-the-environment)
- [간단한 환경 만들기](#creating-a-simple-environment)
- [기대 SARSA 알고리즘 구현하기](#implementing-the-expected-sarsa-algorithm)
- [Q-테이블 초기화 및 업데이트](#q-table-initialization-and-updates)
- [탐험 대 활용 전략](#exploration-vs-exploitation-strategy)
- [기대 SARSA 알고리즘 실행하기](#running-the-expected-sarsa-algorithm)
- [학습 과정 시각화](#visualizing-the-learning-process)
- [Q-값 및 최적 정책 분석](#analyzing-q-values-and-optimal-policy)
- [다른 하이퍼파라미터로 테스트하기 (선택 사항)](#testing-with-different-hyperparameters-optional)
- [다른 환경에 기대 SARSA 적용하기 (절벽 걷기)](#applying-expected-sarsa-to-different-environments-cliff-walking)
- [일반적인 문제점과 해결책](#common-challenges-and-solutions)
- [기대 SARSA vs. 다른 강화 학습 알고리즘](#expected-sarsa-vs-other-reinforcement-learning-algorithms)
  - [기대 SARSA의 장점](#advantages-of-expected-sarsa)
  - [기대 SARSA의 한계점](#limitations-of-expected-sarsa)
  - [관련 알고리즘](#related-algorithms)
- [결론](#conclusion)

## 기대 SARSA란?

기대 SARSA는 강화 학습에서 **온-폴리시(on-policy)** 시간차(TD) 제어 알고리즘입니다. 이는 표준 SARSA 알고리즘의 개선된 버전입니다. SARSA는 현재 Q-값을 업데이트하기 위해 정책에 의해 선택된 *특정* 다음 상태-행동 쌍($Q(s', a')$)의 Q-값을 사용하는 반면, 기대 SARSA는 현재 정책에 따른 확률로 가중된 *모든 가능한* 다음 행동에 대한 *기대* Q-값을 사용합니다.

이러한 기대값 사용은 일반적으로 SARSA에 비해 업데이트 분산을 낮추는 경향이 있습니다. 이는 탐험적이거나 차선책일 수 있는 단일 샘플링된 다음 행동에 의존하지 않기 때문입니다. 이는 일부 환경에서 더 빠르고 안정적인 학습으로 이어질 수 있습니다.

## 기대 SARSA는 어디서 어떻게 사용되는가

기대 SARSA는 SARSA 및 Q-러닝과 많은 응용 프로그램을 공유하지만 업데이트 분산을 줄이는 것이 유익한 경우 잠재적인 이점을 제공합니다:

1.  **확률적 환경**: 행동의 결과나 보상이 노이즈가 많은 경우, 가능한 다음 행동에 대한 평균을 내면 더 부드러운 학습으로 이어질 수 있습니다.
2.  **로보틱스 및 제어**: SARSA와 유사하게 안전한 정책을 학습하는 데 유용하며, 낮은 분산으로 인해 잠재적으로 더 빠른 수렴이 가능합니다.
3.  **SARSA가 적용 가능하지만 업데이트 분산이 높은 모든 시나리오.**

기대 SARSA는 SARSA와 유사한 조건에서 잘 작동합니다:
- 테이블 형태에서 이산 상태 및 행동 공간.
- 온-폴리시 학습 방식이 필요한 경우.
- 환경 상태의 완전한 관찰 가능성.

## 기대 SARSA의 수학적 기초

### 복잡한 원본 버전

기대 SARSA 알고리즘은 다음 업데이트 규칙을 사용하여 Q-값을 업데이트합니다:

$$Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[r_t + \gamma \mathbb{E}_{\pi}[Q(s_{t+1}, A')] - Q(s_t, a_t)\right]$$

이는 다음과 같이 확장됩니다:

$$Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[r_t + \gamma \sum_{a'} \pi(a'|s_{t+1}) Q(s_{t+1}, a') - Q(s_t, a_t)\right]$$

여기서:
- $Q(s_t, a_t)$는 상태 $s_t$와 행동 $a_t$에 대한 Q-값입니다.
- $\alpha$는 학습률입니다 (0 < $\alpha$ ≤ 1).
- $r_t$는 상태 $s_t$에서 행동 $a_t$를 취한 후 받은 보상입니다.
- $\gamma$는 할인 인자입니다 (0 ≤ $\gamma$ ≤ 1).
- $s_{t+1}$은 행동 $a_t$를 취한 후 관찰된 다음 상태입니다.
- $\sum_{a'} \pi(a'|s_{t+1}) Q(s_{t+1}, a')$는 다음 상태 $s_{t+1}$에서의 **기대 Q-값**입니다. 이는 현재 정책 $\pi$에 따라 상태 $s_{t+1}$에서 각 행동 $a'$를 취할 확률 $\pi(a'|s_{t+1})$로 가중된 모든 가능한 다음 행동 $a'$에 대한 Q-값의 합입니다.
- 대괄호 안의 항은 현재 정책 하에서의 다음 상태의 *기대* 값에 기반한 TD 오차입니다.

만약 정책 $\pi$가 현재 Q-값에 대해 엡실론-그리디라면:
- $a^*_{s'} = \arg\max_{a''} Q(s', a'')$를 상태 $s'$에서의 탐욕적 행동이라고 합시다.
- 탐욕적 행동을 선택할 확률은 $\pi(a^*_{s'}|s') = 1 - \epsilon + \frac{\epsilon}{|\mathcal{A}|}$입니다.
- 탐욕적이지 않은 행동 $a' \neq a^*_{s'}$를 선택할 확률은 $\pi(a'|s') = \frac{\epsilon}{|\mathcal{A}|}$입니다.
- 여기서 $|\mathcal{A}|$는 상태 $s'$에서 가능한 행동의 수입니다.

기대값 항은 다음과 같이 됩니다:
$$ \mathbb{E}_{\pi}[Q(s', A')] = (1 - \epsilon + \frac{\epsilon}{|\mathcal{A}|}) Q(s', a^*_{s'}) + \sum_{a' \neq a^*_{s'}} \frac{\epsilon}{|\mathcal{A}|} Q(s', a') $$
$$ = (1 - \epsilon) Q(s', a^*_{s'}) + \frac{\epsilon}{|\mathcal{A}|} \sum_{a'} Q(s', a') $$

### 간단한 버전

더 간단한 해석:

$$
Q_{\text{new}} = Q_{\text{old}} + \alpha \left[ R + \gamma E[Q_{\text{next}}] - Q_{\text{old}} \right]
$$

여기서 $E[Q_{\text{next}}]$는 다음 상태에서의 기대 Q-값으로, 정책의 확률에 따라 모든 가능한 행동에 대해 평균을 내어 계산됩니다.

또는:

$$
Q_{\text{new}} = Q_{\text{old}} + \alpha \left[ \text{Target} - Q_{\text{old}} \right]
$$

여기서 "Target"은 $R + \gamma E[Q_{\text{next}}]$입니다.

## 기대 SARSA 단계별 설명

1.  **Q-테이블 초기화**: 모든 상태 s와 행동 a에 대해 Q(s, a) 테이블을 생성합니다 (일반적으로 0으로 초기화).
2.  **각 에피소드에 대해 반복**:
    a.  상태 $s$를 초기화합니다.
    b.  **에피소드의 각 스텝에 대해 반복**:
        i.  Q로부터 파생된 정책(예: 엡실론-그리디)을 사용하여 상태 $s$에서 행동 $a$를 선택합니다.
        ii. 행동 $a$를 취하고, 보상 $r$과 다음 상태 $s'$를 관찰합니다.
        iii. **$s'$에 대한 기대 Q-값 계산**: $E[Q(s', A')] = \sum_{a'} \pi(a'|s') Q(s', a')$를 계산합니다.
        iv. **Q-값 업데이트**: 기대 SARSA 업데이트 규칙을 적용합니다:
            $Q(s, a) \leftarrow Q(s, a) + \alpha [r + \gamma E[Q(s', A')] - Q(s, a)]$
        v.  상태 업데이트: $s \leftarrow s'$.
        vi. $s$가 종료 상태이면 에피소드를 종료합니다.
3.  **반복**: Q-값이 수렴하거나 최대 에피소드 수에 도달할 때까지 에피소드 실행을 계속합니다.

*참고*: SARSA와 달리, 상태 $s'$에서 선택된 행동(SARSA에서는 $a'$가 됨)은 $Q(s, a)$ 업데이트에 직접 사용되지 않습니다. 업데이트는 $s'$에서 모든 행동에 대한 *기대값*을 사용합니다. 행동 $a'$는 여전히 다음 스텝의 *다음* 상태 전이를 결정하기 위해 선택됩니다.

## 기대 SARSA의 핵심 요소

### Q-테이블
Q-러닝 및 SARSA와 동일한 구조로, 상태-행동 값을 저장합니다.

### 탐험 대 활용 (정책)
기대 SARSA는 온-폴리시이며, 이는 업데이트에 사용되는 정책이 행동 생성에 사용되는 정책과 동일하다는 것을 의미합니다. **엡실론-그리디** 전략이 일반적으로 사용됩니다. 기대값 계산은 각 행동의 확률을 결정하기 위해 엡실론 값과 Q-값을 명시적으로 사용합니다.

### 학습률 (α)
Q-값 업데이트의 스텝 크기를 제어합니다. Q-러닝/SARSA와 동일한 역할을 합니다.

### 할인 인자 (γ)
미래 보상의 현재 가치를 결정합니다. Q-러닝/SARSA와 동일한 역할을 합니다.

## 기대 SARSA vs. SARSA vs. Q-러닝

| 특징              | Q-러닝                                           | SARSA                                      | 기대 SARSA                                                              |
| :---------------- | :--------------------------------------------------- | :----------------------------------------- | :-------------------------------------------------------------------------- |
| **유형**          | 오프-폴리시 (Off-Policy)                           | 온-폴리시 (On-Policy)                      | 온-폴리시 (On-Policy)                                                       |
| **업데이트 목표** | $r + \gamma \max_{a'} Q(s', a')$                      | $r + \gamma Q(s', a')$                     | $r + \gamma \sum_{a'} \pi(a'\mid s') Q(s', a')$ (기대값) |
| **기반**          | 최적 가치 함수 학습                                | 현재 정책의 가치 함수 학습                 | 현재 정책의 가치 함수 학습                                                  |
| **탐험**          | 탐험 선택과 무관하게 최적 경로 학습                   | 업데이트는 탐험적 행동 $a'$에 의존          | 업데이트는 $a'$ 선택으로 인한 분산을 줄이기 위해 탐험에 대해 평균냄              |
| **분산**          | 높을 수 있음 (max 연산자)                          | 높음 (단일 샘플링된 $a'$에 의존)             | SARSA보다 낮음 (기대값 사용)                                                  |
| **편향**          | 잠재적 최대화 편향                                   | Q-러닝보다 편향이 적음                     | Q-러닝보다 편향이 적음                                                      |
| **행동**          | 더 공격적/최적일 수 있음                           | 종종 더 보수적/안전함                       | 종종 SARSA보다 안정적/부드러운 학습, SARSA와 유사한 행동                     |

# 환경 설정
수치 연산을 위한 NumPy와 시각화를 위한 Matplotlib 등 필요한 라이브러리를 임포트합니다.

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np  # 수치 연산을 위해
import matplotlib.pyplot as plt  # 시각화를 위해

# 타입 힌트 임포트
from typing import List, Tuple, Dict, Optional

# 재현성을 위한 시드 설정
np.random.seed(42)

# Jupyter Notebook용 인라인 플로팅 활성화
%matplotlib inline

# 간단한 환경 만들기

기대 SARSA 알고리즘을 위한 간단한 환경을 만들기 위해 4x4 그리드월드를 정의합니다. 그리드월드는 다음과 같은 속성을 가집니다:

- 4 행과 4 열
- 가능한 행동: '위', '아래', '왼쪽', '오른쪽'
- 보상이 있는 특정 종료 상태.
- 절벽 상태 (나중에 절벽 걷기 예제에서).

In [ ]:
# 그리드월드 환경 생성 함수 정의
def create_gridworld(
    rows: int,
    cols: int,
    terminal_states: List[Tuple[int, int]],
    rewards: Dict[Tuple[int, int], int]
) -> Tuple[np.ndarray, List[Tuple[int, int]], List[str]]:
    """
    간단한 그리드월드 환경을 생성합니다.

    Parameters:
    - rows (int): 그리드의 행 수.
    - cols (int): 그리드의 열 수.
    - terminal_states (List[Tuple[int, int]]): (행, 열) 튜플로 구성된 종료 상태 리스트.
    - rewards (Dict[Tuple[int, int], int]): (행, 열)을 보상 값에 매핑하는 딕셔너리.

    Returns:
    - grid (np.ndarray): 보상을 포함하는 그리드를 나타내는 2D 배열 (참고용, 에이전트가 사용하지 않음).
    - state_space (List[Tuple[int, int]]): 그리드 내 모든 가능한 상태 리스트.
    - action_space (List[str]): 가능한 행동 리스트 ('up', 'down', 'left', 'right').
    """
    # 그리드를 0으로 초기화 (시각화/참고용)
    grid = np.zeros((rows, cols))

    # 지정된 상태에 보상 할당
    for (row, col), reward in rewards.items():
        grid[row, col] = reward

    # 상태 공간을 모든 가능한 (행, 열) 쌍으로 정의
    state_space = [
        (row, col)
        for row in range(rows)
        for col in range(cols)
    ]

    # 행동 공간을 네 가지 가능한 이동으로 정의
    action_space = ['up', 'down', 'left', 'right']

    return grid, state_space, action_space

다음으로 상태 전이 함수가 필요합니다. 이 함수는 현재 상태와 행동을 입력받아 다음 상태를 반환합니다. 이는 에이전트가 취하는 행동에 따라 그리드 주위를 움직이는 것으로 생각할 수 있습니다. 이 경우 환경은 결정론적입니다.

In [ ]:
# 상태 전이 함수 정의
def state_transition(state: Tuple[int, int], action: str, rows: int, cols: int) -> Tuple[int, int]:
    """
    현재 상태와 행동을 고려하여 다음 상태를 계산합니다. 경계를 처리합니다.

    Parameters:
    - state (Tuple[int, int]): 현재 상태 (행, 열).
    - action (str): 취할 행동 ('up', 'down', 'left', 'right').
    - rows (int): 그리드의 행 수.
    - cols (int): 그리드의 열 수.

    Returns:
    - Tuple[int, int]: 행동을 취한 후의 결과 상태 (행, 열).
    """
    # 현재 상태를 행과 열로 언패킹
    row, col = state
    next_row, next_col = row, col

    # 행동에 따라 행 또는 열을 업데이트하고 경계를 존중하도록 보장
    if action == 'up' and row > 0:  # 최상단 행이 아니면 위로 이동
        next_row -= 1
    elif action == 'down' and row < rows - 1:  # 최하단 행이 아니면 아래로 이동
        next_row += 1
    elif action == 'left' and col > 0:  # 최좌측 열이 아니면 왼쪽으로 이동
        next_col -= 1
    elif action == 'right' and col < cols - 1:  # 최우측 열이 아니면 오른쪽으로 이동
        next_col += 1
    # 행동이 그리드 밖으로 이동하면 같은 상태에 머무름 (행, 열 변경 없음)

    # 새 상태를 튜플로 반환
    return (next_row, next_col)

이제 에이전트가 환경과 상호 작용할 수 있으므로 보상 함수를 정의해야 합니다. 이 함수는 주어진 상태에 도달했을 때의 보상을 반환하며, 이는 훈련 중 Q-값을 업데이트하는 데 사용됩니다.

In [ ]:
# 보상 함수 정의
def get_reward(state: Tuple[int, int], rewards: Dict[Tuple[int, int], int]) -> int:
    """
    주어진 상태에 대한 보상을 얻습니다.

    Parameters:
    - state (Tuple[int, int]): 현재 상태 (행, 열).
    - rewards (Dict[Tuple[int, int], int]): 상태 (행, 열)를 보상 값에 매핑하는 딕셔너리.

    Returns:
    - int: 주어진 상태에 대한 보상. 상태가 보상 딕셔너리에 없으면 0을 반환합니다.
    """
    # rewards 딕셔너리를 사용하여 주어진 상태에 대한 보상을 가져옵니다.
    # 상태를 찾을 수 없으면 기본 보상 0을 반환합니다.
    return rewards.get(state, 0)

이제 그리드월드 환경과 필요한 도우미 함수를 정의했으므로 간단한 예제로 테스트해 보겠습니다. (0, 0)과 (3, 3)에 각각 1과 10의 보상을 갖는 두 개의 종료 상태가 있는 4x4 그리드를 생성합니다. 그런 다음 상태 (2, 2)에서 위쪽으로 이동하여 상태 전이 및 보상 함수를 테스트합니다.

In [ ]:
# 그리드월드 환경 예제 사용법

# 그리드 차원 (4x4), 종료 상태 및 보상 정의
rows, cols = 4, 4  # 그리드의 행 및 열 수
terminal_states = [(0, 0), (3, 3)]  # 종료 상태
rewards = {(0, 0): 1, (3, 3): 10}  # 종료 상태에 대한 보상 (다른 상태는 보상 0)

# 그리드월드 환경 생성
grid, state_space, action_space = create_gridworld(rows, cols, terminal_states, rewards)

# 상태 전이 및 보상 함수 테스트
current_state = (2, 2)  # 시작 상태
action = 'up'  # 취할 행동
next_state = state_transition(current_state, action, rows, cols)  # 다음 상태 계산
reward = get_reward(next_state, rewards)  # 다음 상태에 대한 보상 얻기

# 결과 출력
print("그리드월드 (보상 보기):")  # 보상이 있는 그리드 표시
print(grid)
print(f"현재 상태: {current_state}")  # 현재 상태 표시
print(f"취한 행동: {action}")  # 취한 행동 표시
print(f"다음 상태: {next_state}")  # 결과 다음 상태 표시
print(f"다음 상태에서의 보상: {reward}")  # 다음 상태에 대한 보상 표시

지정된 차원, 종료 상태 및 보상으로 그리드월드 환경이 생성된 것을 볼 수 있습니다. 시작 상태 (2, 2)와 행동('위')을 선택했습니다. 다음 상태는 (1, 2)로 계산되고, 상태 (1, 2)에 도달하는 보상은 지정된 보상 상태가 아니므로 0입니다.

# 기대 SARSA 알고리즘 구현하기

그리드월드 환경을 성공적으로 구현했습니다. 이제 핵심 구성 요소인 Q-테이블 초기화, 엡실론-그리디 정책 및 특정 기대 SARSA 업데이트 규칙을 구현합니다.

In [ ]:
# Q-테이블 초기화
def initialize_q_table(state_space: List[Tuple[int, int]], action_space: List[str]) -> Dict[Tuple[Tuple[int, int], str], float]:
    """
    모든 상태-행동 쌍에 대해 Q-테이블을 0으로 초기화합니다.
    (상태, 행동) 튜플을 키로 사용하는 단일 딕셔너리를 사용합니다.

    Parameters:
    - state_space (List[Tuple[int, int]]): 모든 가능한 상태 리스트.
    - action_space (List[str]): 모든 가능한 행동 리스트.

    Returns:
    - q_table (Dict[Tuple[Tuple[int, int], str], float]): (상태, 행동) 쌍을 Q-값에 매핑하는 딕셔너리, 0.0으로 초기화됨.
    """
    q_table: Dict[Tuple[Tuple[int, int], str], float] = {}
    for state in state_space:
        for action in action_space:
            # (상태, 행동) 쌍에 대한 Q-값을 0.0으로 초기화
            q_table[(state, action)] = 0.0
    return q_table

In [ ]:
# --- 대안 Q-테이블 구조 (중첩 딕셔너리) ---
# 더 쉬운 비교와 재사용을 위해. run_sarsa_episode 함수가 적응할 것입니다.
def initialize_q_table_nested(state_space: List[Tuple[int, int]], action_space: List[str]) -> Dict[Tuple[int, int], Dict[str, float]]:
    """
    중첩 딕셔너리 구조를 사용하여 Q-테이블을 0으로 초기화합니다.

    Parameters:
    - state_space (List[Tuple[int, int]]): 모든 가능한 상태 리스트.
    - action_space (List[str]): 모든 가능한 행동 리스트.

    Returns:
    - q_table (Dict[Tuple[int, int], Dict[str, float]]): q_table[state][action]이 Q-값을 제공하는 중첩 딕셔너리.
    """
    q_table: Dict[Tuple[int, int], Dict[str, float]] = {}
    for state in state_space:
        # 종료 상태의 경우, 해당 상태에서 행동을 취할 수 없으므로 Q-값은 0으로 유지되어야 합니다.
        # 그러나 종료 확인 전에 업데이트 중 조회를 용이하게 하기 위해 모두 초기화합니다.
        q_table[state] = {action: 0.0 for action in action_space}
    return q_table

다음으로, 행동 선택을 위한 엡실론-그리디 정책을 구현합니다. 이 정책은 엡실론 확률로 무작위 행동을 선택하고 1 - 엡실론 확률로 최상의 행동(가장 높은 Q-값)을 선택하여 탐험과 활용의 균형을 맞춥니다.

In [ ]:
# 엡실론-그리디 정책을 사용하여 행동 선택
def epsilon_greedy_policy(
    state: Tuple[int, int],  # 현재 상태 (행, 열) 튜플
    q_table: Dict[Tuple[int, int], Dict[str, float]],  # 중첩 딕셔너리 형태의 Q-테이블
    action_space: List[str],  # 가능한 행동 리스트
    epsilon: float  # 탐험률 (무작위 행동을 선택할 확률)
) -> str:
    """
    엡실론-그리디 정책을 사용하여 행동을 선택합니다.

    Parameters:
    - state (Tuple[int, int]): 에이전트의 현재 상태.
    - q_table (Dict[Tuple[int, int], Dict[str, float]]): 상태를 행동 및 해당 Q-값에 매핑하는 Q-테이블.
    - action_space (List[str]): 모든 가능한 행동 리스트.
    - epsilon (float): 탐험률 (0 <= epsilon <= 1).

    Returns:
    - str: 선택된 행동.
    """
    # 상태가 Q-테이블에 없으면 무작위 행동 선택
    if state not in q_table:
        return np.random.choice(action_space)

    # 엡실론 확률로 무작위 행동 선택 (탐험)
    if np.random.rand() < epsilon:
        return np.random.choice(action_space)
    else:
        # 그렇지 않으면 가장 높은 Q-값을 가진 행동 선택 (활용)
        if q_table[state]:  # 상태에 유효한 Q-값이 있는지 확인
            # 현재 상태에 대한 최대 Q-값 찾기
            max_q = max(q_table[state].values())
            # 최대 Q-값을 가진 모든 행동 찾기 (동점 처리)
            best_actions = [action for action, q in q_table[state].items() if q == max_q]
            # 동점일 경우 최상의 행동 중에서 무작위로 선택
            return np.random.choice(best_actions)
        else:
            # 상태에 사용 가능한 Q-값이 없으면 무작위 행동 선택
            return np.random.choice(action_space)

에이전트가 기대 SARSA 알고리즘을 사용하여 Q-값을 업데이트하는 로직을 코딩해 보겠습니다. 이는 Q-테이블을 사용하여 현재 정책(엡실론-그리디)에 기반한 다음 상태의 기대값을 계산합니다. Q-값 업데이트는 받은 보상과 다음 상태의 기대값을 기반으로 하며, 학습률(알파)과 할인 인자(감마)로 조정됩니다.

In [ ]:
# 기대 SARSA 규칙을 사용하여 Q-값 업데이트
def update_expected_sarsa_value(
    q_table: Dict[Tuple[int, int], Dict[str, float]],  # 상태를 행동 및 Q-값에 매핑하는 Q-테이블
    state: Tuple[int, int],  # 현재 상태 (행, 열) 튜플
    action: str,  # 현재 상태에서 취한 행동
    reward: int,  # 행동을 취한 후 받은 보상
    next_state: Tuple[int, int],  # 행동을 취한 후 도달한 다음 상태
    alpha: float,  # 학습률 (스텝 크기)
    gamma: float,  # 할인 인자 (미래 보상의 중요도)
    epsilon: float,  # 탐험률 (정책 확률 계산에 사용됨)
    action_space: List[str],  # 모든 가능한 행동 리스트
    terminal_states: List[Tuple[int, int]]  # 종료 상태 리스트
) -> None:
    """
    기대 SARSA 규칙을 사용하여 주어진 상태-행동 쌍에 대한 Q-값을 업데이트합니다.

    Parameters:
    - q_table: 모든 상태-행동 쌍에 대한 Q-값을 저장하는 Q-테이블.
    - state: 현재 상태 (행, 열) 튜플.
    - action: 현재 상태에서 취한 행동.
    - reward: 행동을 취한 후 받은 보상.
    - next_state: 행동을 취한 후 도달한 다음 상태.
    - alpha: 학습률 (0 < alpha <= 1).
    - gamma: 할인 인자 (0 <= gamma <= 1).
    - epsilon: 탐험률 (0 <= epsilon <= 1).
    - action_space: 모든 가능한 행동 리스트.
    - terminal_states: 환경의 종료 상태 리스트.

    Returns:
    - None: Q-테이블의 Q-값을 제자리에서 업데이트합니다.
    """
    # 현재 상태와 행동이 Q-테이블에 존재하는지 확인
    if state not in q_table or action not in q_table[state]:
        return

    # 다음 상태에 대한 기대 Q-값 초기화
    expected_q_next: float = 0.0

    # 다음 상태가 종료 상태가 아니고 Q-테이블에 존재하며 유효한 Q-값을 가지면 기대 Q-값 계산
    if next_state not in terminal_states and next_state in q_table and q_table[next_state]:
        # 다음 상태의 모든 행동에 대한 Q-값 가져오기
        q_values_next_state: Dict[str, float] = q_table[next_state]

        # 다음 상태에서 최대 Q-값 찾기
        max_q_next: float = max(q_values_next_state.values())

        # 최대 Q-값을 가진 모든 행동 식별 (탐욕적 행동)
        greedy_actions: List[str] = [a for a, q in q_values_next_state.items() if q == max_q_next]

        # 탐욕적 및 비탐욕적 행동에 대한 확률 계산
        num_actions: int = len(action_space)
        num_greedy_actions: int = len(greedy_actions)
        prob_greedy: float = (1.0 - epsilon) / num_greedy_actions + epsilon / num_actions
        prob_non_greedy: float = epsilon / num_actions

        # 다음 상태에 대한 기대 Q-값 계산
        for a_prime in action_space:
            q_s_prime_a_prime: float = q_values_next_state.get(a_prime, 0.0)
            if a_prime in greedy_actions:
                expected_q_next += prob_greedy * q_s_prime_a_prime
            else:
                expected_q_next += prob_non_greedy * q_s_prime_a_prime

    # 다음 상태가 종료 상태이면 expected_q_next는 0.0으로 유지됨

    # 시간차(TD) 목표 및 오차 계산
    # TD 목표: td_target = reward + gamma * expected_q_next
    # TD 오차: td_error = td_target - q_table[state][action]
    td_target: float = reward + gamma * expected_q_next
    td_error: float = td_target - q_table[state][action]

    # 현재 상태-행동 쌍에 대한 Q-값 업데이트
    # Q(s, a) <- Q(s, a) + alpha * td_error
    q_table[state][action] += alpha * td_error

지금까지 환경과 핵심 기대 SARSA Q-값 업데이트 로직을 정의했습니다. 이제 이것들을 결합하여 기대 SARSA 업데이트 로직을 사용하여 단일 에피소드를 실행하는 함수를 만듭니다.

In [ ]:
# 기대 SARSA의 단일 에피소드 실행
def run_expected_sarsa_episode(
    q_table: Dict[Tuple[int, int], Dict[str, float]],  # 상태를 행동 및 Q-값에 매핑하는 Q-테이블
    state_space: List[Tuple[int, int]],  # 환경의 모든 가능한 상태 리스트
    action_space: List[str],  # 모든 가능한 행동 리스트
    rewards: Dict[Tuple[int, int], int],  # 상태를 보상에 매핑하는 딕셔너리
    terminal_states: List[Tuple[int, int]],  # 환경의 종료 상태 리스트
    rows: int,  # 그리드의 행 수
    cols: int,  # 그리드의 열 수
    alpha: float,  # 학습률 (0 < alpha <= 1)
    gamma: float,  # 할인 인자 (0 <= gamma <= 1)
    epsilon: float,  # 탐험률 (0 <= epsilon <= 1)
    max_steps: int  # 에피소드당 최대 스텝 수
) -> Tuple[int, int]:
    """
    기대 SARSA 업데이트를 사용하여 단일 에피소드를 실행합니다.

    Parameters:
    - q_table: 모든 상태-행동 쌍에 대한 Q-값을 저장하는 Q-테이블.
    - state_space: 환경의 모든 가능한 상태 리스트.
    - action_space: 모든 가능한 행동 리스트.
    - rewards: 상태를 보상에 매핑하는 딕셔너리.
    - terminal_states: 환경의 종료 상태 리스트.
    - rows: 그리드의 행 수.
    - cols: 그리드의 열 수.
    - alpha: 학습률 (0 < alpha <= 1).
    - gamma: 할인 인자 (0 <= gamma <= 1).
    - epsilon: 탐험률 (0 <= epsilon <= 1).
    - max_steps: 에피소드당 허용되는 최대 스텝 수.

    Returns:
    - Tuple[int, int]: 에피소드 동안 누적된 총 보상과 취한 스텝 수.
    """
    # 시작 상태를 무작위로 초기화하되, 종료 상태가 아닌지 확인
    state: Tuple[int, int] = state_space[np.random.choice(len(state_space))]
    while state in terminal_states:
        state = state_space[np.random.choice(len(state_space))]

    total_reward: int = 0  # 에피소드의 총 보상 누적
    steps: int = 0  # 에피소드에서 취한 스텝 수 추적

    for _ in range(max_steps):
        # 엡실론-그리디 정책을 사용하여 행동 선택
        action: str = epsilon_greedy_policy(state, q_table, action_space, epsilon)

        # 선택된 행동을 취하고 다음 상태와 보상 관찰
        next_state: Tuple[int, int] = state_transition(state, action, rows, cols)
        reward: int = get_reward(next_state, rewards)
        total_reward += reward  # 총 보상 업데이트

        # 기대 SARSA를 사용하여 현재 상태-행동 쌍에 대한 Q-값 업데이트
        update_expected_sarsa_value(
            q_table, state, action, reward, next_state, alpha, gamma, epsilon, action_space, terminal_states
        )

        # 다음 상태로 이동
        state = next_state
        steps += 1  # 스텝 카운터 증가

        # 종료 상태에 도달하면 에피소드 종료
        if state in terminal_states:
            break

    return total_reward, steps

# 탐험 대 활용 전략

탐험과 활용의 균형을 적절히 맞추기 위해 이전에 정의된 엡실론-그리디 정책을 사용하고 동적 엡실론 조정을 구현합니다. 엡실론은 높은 값(더 많은 탐험)에서 시작하여 점진적으로 감소하여 에이전트가 학습함에 따라 더 많은 활용을 장려합니다.

In [ ]:
# 동적 엡실론 조정 함수 정의 (Q-러닝/SARSA 참조와 동일)
def adjust_epsilon(
    initial_epsilon: float,
    min_epsilon: float,
    decay_rate: float,
    episode: int
) -> float:
    """
    지수 감쇠를 사용하여 시간이 지남에 따라 엡실론을 동적으로 조정합니다.

    Parameters:
    - initial_epsilon (float): 초기 탐험률.
    - min_epsilon (float): 최소 탐험률.
    - decay_rate (float): 엡실론이 감쇠하는 비율.
    - episode (int): 현재 에피소드 번호.

    Returns:
    - float: 현재 에피소드에 대해 조정된 탐험률.
    """
    # 감쇠된 엡실론 값을 계산하되, 최소 엡실론 아래로 내려가지 않도록 보장
    return max(min_epsilon, initial_epsilon * np.exp(-decay_rate * episode))

계획된 에피소드에 걸쳐 엡실론 감쇠를 추적하고 플로팅하여 탐험 전략을 시각화해 봅시다.

In [ ]:
# 동적 엡실론 조정 사용 예 및 감쇠 플로팅

# 엡실론 파라미터 정의
initial_epsilon: float = 1.0  # 완전 탐험으로 시작
min_epsilon: float = 0.1  # 최소 탐험률
decay_rate: float = 0.01  # 엡실론 감쇠율
episodes: int = 500  # 훈련을 위한 총 에피소드 수

# 에피소드에 걸쳐 엡실론 값 추적
epsilon_values: List[float] = []
for episode in range(episodes):
    # 현재 에피소드에 대한 엡실론 조정
    current_epsilon = adjust_epsilon(initial_epsilon, min_epsilon, decay_rate, episode)
    epsilon_values.append(current_epsilon)
    

# 에피소드에 걸쳐 엡실론 감쇠 플로팅
plt.figure(figsize=(20, 3)) # 그림 크기 약간 조정
plt.plot(epsilon_values)
plt.xlabel('에피소드')  # x축 레이블
plt.ylabel('엡실론')  # y축 레이블
plt.title('에피소드에 따른 엡실론 감쇠')  # 플롯 제목
plt.grid(True) # 가독성 향상을 위해 그리드 추가
plt.show()  # 플롯 표시

이 플롯은 엡실론이 1.0(순수 탐험)에서 시작하여 최소값 0.1(주로 활용하며 10% 탐험)을 향해 지수적으로 감쇠하는 것을 보여줍니다. 이러한 점진적인 변화는 에이전트가 초기에 환경을 탐색하고 학습한 내용을 바탕으로 정책을 개선할 수 있도록 합니다.

# 기대 SARSA 알고리즘 실행하기

이제 그리드월드 환경에서 여러 에피소드에 걸쳐 기대 SARSA 알고리즘을 실행하고, 동적 엡실론 조정을 사용합니다. 각 에피소드에 대한 총 보상과 에피소드 길이를 추적합니다.

In [ ]:
# 여러 에피소드에 걸쳐 기대 SARSA를 실행하는 함수
def run_expected_sarsa(
    state_space: List[Tuple[int, int]],  # 환경의 모든 가능한 상태 리스트
    action_space: List[str],  # 모든 가능한 행동 리스트
    rewards: Dict[Tuple[int, int], int],  # 상태를 보상에 매핑하는 딕셔너리
    terminal_states: List[Tuple[int, int]],  # 환경의 종료 상태 리스트
    rows: int,  # 그리드의 행 수
    cols: int,  # 그리드의 열 수
    alpha: float,  # 학습률 (0 < alpha <= 1)
    gamma: float,  # 할인 인자 (0 <= gamma <= 1)
    initial_epsilon: float,  # 초기 탐험률 (0 <= epsilon <= 1)
    min_epsilon: float,  # 최소 탐험률
    decay_rate: float,  # 엡실론 감쇠율
    episodes: int,  # 훈련할 에피소드 수
    max_steps: int  # 에피소드당 최대 스텝 수
) -> Tuple[Dict[Tuple[int, int], Dict[str, float]], List[int], List[int]]:
    """
    여러 에피소드에 걸쳐 기대 SARSA 알고리즘을 실행합니다.

    Parameters:
    - state_space: 환경의 모든 가능한 상태 리스트.
    - action_space: 모든 가능한 행동 리스트.
    - rewards: 상태를 보상에 매핑하는 딕셔너리.
    - terminal_states: 환경의 종료 상태 리스트.
    - rows: 그리드의 행 수.
    - cols: 그리드의 열 수.
    - alpha: 학습률 (0 < alpha <= 1).
    - gamma: 할인 인자 (0 <= gamma <= 1).
    - initial_epsilon: 초기 탐험률 (0 <= epsilon <= 1).
    - min_epsilon: 최소 탐험률.
    - decay_rate: 엡실론 감쇠율.
    - episodes: 훈련할 에피소드 수.
    - max_steps: 에피소드당 최대 스텝 수.

    Returns:
    - Tuple containing:
        - q_table: 상태를 행동 및 해당 Q-값에 매핑하는 중첩 딕셔너리.
        - rewards_per_episode: 각 에피소드에서 누적된 총 보상 리스트.
        - episode_lengths: 각 에피소드에서 취한 스텝 수 리스트.
    """
    # 모든 상태-행동 쌍에 대해 Q-테이블을 0으로 초기화
    q_table: Dict[Tuple[int, int], Dict[str, float]] = initialize_q_table_nested(state_space, action_space)

    # 각 에피소드의 보상 및 에피소드 길이를 저장할 리스트
    rewards_per_episode: List[int] = []
    episode_lengths: List[int] = []

    # 각 에피소드 루프
    for episode in range(episodes):
        # 현재 에피소드에 대한 엡실론 동적 조정
        epsilon: float = adjust_epsilon(initial_epsilon, min_epsilon, decay_rate, episode)

        # 기대 SARSA의 단일 에피소드 실행
        total_reward, steps = run_expected_sarsa_episode(
            q_table, state_space, action_space, rewards, terminal_states,
            rows, cols, alpha, gamma, epsilon, max_steps
        )

        # 총 보상 및 에피소드 길이 저장
        rewards_per_episode.append(total_reward)
        episode_lengths.append(steps)

    # Q-테이블, 에피소드별 보상, 에피소드 길이 반환
    return q_table, rewards_per_episode, episode_lengths

하이퍼파라미터를 설정하고 SARSA 훈련 프로세스를 실행합니다.

In [ ]:
# --- 기대 SARSA 실행 ---
# 기대 SARSA 하이퍼파라미터 정의
alpha_es = 0.1  # 학습률: Q-값 업데이트의 스텝 크기 제어
gamma_es = 0.9  # 할인 인자: 미래 보상의 중요도 결정
initial_epsilon_es = 1.0  # 초기 탐험률: 무작위 행동 선택 확률
min_epsilon_es = 0.1  # 최소 탐험률: 엡실론의 하한값
decay_rate_es = 0.01  # 엡실론 감쇠율: 엡실론 감소 속도 제어
episodes_es = 500  # 에이전트 훈련 에피소드 수
max_steps_es = 100  # 에피소드당 허용되는 최대 스텝 수

# 훈련 시작을 알리는 메시지 출력
print("기대 SARSA 실행 중...")

# 지정된 하이퍼파라미터로 기대 SARSA 알고리즘 실행
es_q_table, es_rewards_per_episode, es_episode_lengths = run_expected_sarsa(
    state_space, action_space, rewards, terminal_states, rows, cols, alpha_es, gamma_es,
    initial_epsilon_es, min_epsilon_es, decay_rate_es, episodes_es, max_steps_es
)

# 훈련 완료를 알리는 메시지 출력
print("기대 SARSA 훈련 완료.")

# 학습 과정 시각화

에피소드별 총 보상과 각 에피소드의 길이를 플로팅하여 훈련 진행 상황을 시각화해 봅시다.

In [ ]:
# 기대 SARSA 결과 플로팅
plt.figure(figsize=(20, 3))

# 보상
plt.subplot(1, 2, 1)
plt.plot(es_rewards_per_episode)
plt.xlabel('에피소드')
plt.ylabel('총 보상')
plt.title('기대 SARSA: 에피소드별 보상')
plt.grid(True)

# 에피소드 길이
plt.subplot(1, 2, 2)
plt.plot(es_episode_lengths)
plt.xlabel('에피소드')
plt.ylabel('에피소드 길이')
plt.title('기대 SARSA: 에피소드별 길이')
plt.grid(True)

plt.tight_layout()
plt.show()

**기대 SARSA 학습 곡선 분석**

**보상 곡선 (왼쪽)**
- 보상 곡선은 에피소드 전반에 걸쳐 매우 변동성이 큰 것으로 나타나며, 이는 에이전트 성능의 빈번한 변동을 나타냅니다.
- 부드러운 수렴 대신 보상이 낮은 값과 높은 값 사이에서 진동하며, 이는 에이전트가 때때로 차선의 경로를 따르거나 광범위하게 탐험함을 시사합니다.
- 불안정성은 높은 탐험률, 불충분하게 조정된 학습률 또는 환경의 변동성 때문일 수 있습니다.
- 표준 SARSA와 비교할 때, 기대값 업데이트는 이 경우 변동을 크게 완화하지 못하는 것으로 보입니다.

**에피소드 길이 곡선 (오른쪽)**
- 처음에는 에피소드 길이가 길어, 에이전트가 비효율적인 전략으로 시작함을 확인합니다.
- 시간이 지남에 따라 에피소드 길이가 감소하고 낮은 값에서 안정화되어 에이전트가 더 최적의 경로를 학습하고 있음을 나타냅니다.
- 전반적인 추세는 하향으로 성공적인 학습을 시사하지만, 후반 에피소드에서도 일부 분산이 남아 있습니다.
- 보상 곡선과 비교할 때, 이 플롯은 더 짧은 에피소드 길이가 더 효율적인 목표 도달 행동에 해당하므로 학습 진행 상황을 더 명확하게 보여줍니다.

# Q-값 및 최적 정책 분석

이제 기대 SARSA 알고리즘에서 학습된 Q-값과 그 결과로 도출된 정책을 시각화합니다. 행동별 Q-값에는 히트맵을 사용하고 그리드 상의 정책에는 화살표를 사용합니다.

In [ ]:
# Q-값 히트맵 시각화 함수
def plot_q_values_heatmap(q_table: Dict[Tuple[int, int], Dict[str, float]], rows: int, cols: int, action_space: List[str], fig: plt.Figure, axes: np.ndarray) -> None:
    """
    제공된 축에 각 행동에 대한 Q-값을 히트맵으로 시각화합니다.

    Parameters:
    - q_table: 상태를 행동 및 Q-값에 매핑하는 중첩 딕셔너리.
    - rows: 그리드의 행 수.
    - cols: 그리드의 열 수.
    - action_space: 가능한 행동 리스트 (예: ['up', 'down', 'left', 'right']).
    - fig: Matplotlib figure 객체.
    - axes: 히트맵 플로팅을 위한 Matplotlib axes 배열.
    """
    for i, action in enumerate(action_space):
        # Q-테이블에 없는 상태는 -inf로 그리드 초기화
        q_values = np.full((rows, cols), -np.inf)
        for (row, col), actions in q_table.items():
            if action in actions:
                q_values[row, col] = actions[action]  # 행동에 대한 Q-값 할당

        # 현재 행동에 대한 히트맵 플로팅
        ax = axes[i]
        cax = ax.matshow(q_values, cmap='viridis')  # 'viridis' 컬러맵 사용
        fig.colorbar(cax, ax=ax)  # 히트맵에 컬러바 추가
        ax.set_title(f"기대 SARSA Q-값: {action}")  # 히트맵 제목

        # 더 나은 시각화를 위한 그리드 라인 추가
        ax.set_xticks(np.arange(-.5, cols, 1), minor=True)
        ax.set_yticks(np.arange(-.5, rows, 1), minor=True)
        ax.grid(which='minor', color='w', linestyle='-', linewidth=1)

        # 깔끔한 모양을 위해 틱 레이블 제거
        ax.set_xticks(np.arange(cols))
        ax.set_yticks(np.arange(rows))
        ax.tick_params(axis='both', which='both', length=0)
        ax.set_xticklabels([])
        ax.set_yticklabels([])


# 학습된 정책 시각화 함수
def plot_policy_grid(q_table: Dict[Tuple[int, int], Dict[str, float]], rows: int, cols: int, terminal_states: List[Tuple[int, int]], ax: plt.Axes) -> None:
    """
    제공된 축에 그리드 상의 화살표로 학습된 정책을 시각화합니다.

    Parameters:
    - q_table: 상태를 행동 및 Q-값에 매핑하는 중첩 딕셔너리.
    - rows: 그리드의 행 수.
    - cols: 그리드의 열 수.
    - terminal_states: 그리드의 종료 상태 리스트.
    - ax: 정책 그리드 플로팅을 위한 Matplotlib 축.
    """
    # 정책 기호를 저장할 그리드 초기화
    policy_grid = np.empty((rows, cols), dtype=str)
    action_symbols = {'up': '↑', 'down': '↓', 'left': '←', 'right': '→', '': ''}

    for r in range(rows):
        for c in range(cols):
            state = (r, c)
            if state in terminal_states:
                # 종료 상태를 'T'로 표시
                policy_grid[r, c] = 'T'
                continue
            if state in q_table and q_table[state]:
                # 가장 높은 Q-값을 가진 행동(들) 찾기
                max_q = -np.inf
                best_actions = []
                for action, q_val in q_table[state].items():
                    if q_val > max_q:
                        max_q = q_val
                        best_actions = [action]
                    elif q_val == max_q:
                        best_actions.append(action)

                # 동점일 경우 첫 번째 행동 선택
                if best_actions:
                    best_action = best_actions[0]
                    policy_grid[r, c] = action_symbols[best_action]
                else:
                    # 유효한 행동이 없는 상태 표시
                    policy_grid[r, c] = '.'
            else:
                # 방문하지 않았거나 Q-값이 없는 상태 표시
                policy_grid[r, c] = '.'

    # 정책 그리드 플로팅
    ax.matshow(np.zeros((rows, cols)), cmap='Greys', alpha=0.1)  # 배경 그리드
    for r in range(rows):
        for c in range(cols):
            # 각 셀에 정책 기호 추가
            ax.text(c, r, policy_grid[r, c], ha='center', va='center', fontsize=14, color='black' if policy_grid[r, c] != 'T' else 'red')

    # 그리드 라인 및 제목 추가
    ax.set_title("기대 SARSA 학습된 정책")
    ax.set_xticks(np.arange(-.5, cols, 1), minor=True)
    ax.set_yticks(np.arange(-.5, rows, 1), minor=True)
    ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
# 기대 SARSA의 Q-값 및 정책 시각화
fig_es, axes_es = plt.subplots(1, len(action_space) + 1, figsize=(20, 4))

plot_q_values_heatmap(es_q_table, rows, cols, action_space, fig_es, axes_es[:-1])
axes_es[0].set_title(f"기대 SARSA Q-값: {action_space[0]}") # 제목 조정
axes_es[1].set_title(f"기대 SARSA Q-값: {action_space[1]}")
axes_es[2].set_title(f"기대 SARSA Q-값: {action_space[2]}")
axes_es[3].set_title(f"기대 SARSA Q-값: {action_space[3]}")

plot_policy_grid(es_q_table, rows, cols, terminal_states, axes_es[-1])
axes_es[-1].set_title("기대 SARSA 학습된 정책")


plt.tight_layout()
plt.show()

**기대 SARSA Q-값 및 정책 분석**

**Q-값 히트맵 (왼쪽 네 개 플롯)**
- 이 히트맵들은 모든 그리드 상태에 걸쳐 **각 행동**(위, 아래, 왼쪽, 오른쪽)에 대해 학습된 **Q-값**을 보여줍니다.
- **더 밝은 색상(노란색/녹색)은 더 높은 Q-값**을 나타내며, 이는 특정 상태에서 해당 행동이 더 유리함을 의미합니다.
- **"아래"와 "오른쪽" 행동**은 대부분의 상태에서 가장 높은 Q-값을 가지며, 이는 에이전트를 목표 지점으로 향하게 하는 최적 전략과 일치합니다.
- 특히 종료 상태 근처에서 보상이 인접 상태에 영향을 미치는 **값의 전파**가 보입니다.

**학습된 정책 (가장 오른쪽 플롯)**
- Q-값에서 도출된 **최적 정책은 각 상태에서 최상의 행동 방향을 가리키는 화살표**를 사용하여 시각화됩니다.
- **정책은 주로 오른쪽(→) 및 아래(↓) 이동 패턴**을 따르며, 에이전트를 오른쪽 하단 목표 상태로 효율적으로 안내합니다.
- **위(↑) 및 왼쪽(←) 행동**은 선택된 상태에서 나타나며, 이는 초기 탐험 효과 또는 대안적이지만 차선인 경로의 존재 때문일 가능성이 높습니다.
- **종료 상태('T')**는 빨간색으로 표시된 것처럼 올바르게 학습되었습니다.

**전체 해석:**
- 에이전트는 **Q-값의 명확한 구조와 방향성 화살표**에서 볼 수 있듯이 성공적으로 **안정적인 정책을 학습**했습니다.
- **기대 SARSA는 목표 상태에서 이전 상태로 보상을 효과적으로 전파**하여 최적 경로를 생성했습니다.
- 보다 결정론적인 업데이트 규칙을 따르는 **Q-러닝과 비교할 때, 기대 SARSA의 업데이트는 에이전트의 정책에 영향을 받아 잠재적으로 더 부드러운 학습 과정**으로 이어집니다.
- **최종 Q-값에서는 최소한의 노이즈나 불안정성**이 관찰되어 **좋은 수렴**을 시사합니다.

# Q-값 및 최적 정책 분석
SARSA가 학습한 최적 정책을 표 형식으로 살펴보고, 각 상태에서 각 행동에 대한 Q-값과 해당하는 최상의 행동을 보여줍니다.

In [ ]:
# --- 표 보기 ---
# 각 상태에 대한 Q-값과 최적 행동을 저장할 리스트 생성
es_q_policy_data = []

# 그리드의 각 행과 열 반복
for r in range(rows):
    for c in range(cols):
        state = (r, c)  # 현재 상태를 (행, 열) 튜플로 정의

        # 상태가 Q-테이블에 존재하는지 확인
        if state in es_q_table:
            actions = es_q_table[state]  # 현재 상태의 모든 행동에 대한 Q-값 가져오기

            # 상태에 유효한 Q-값이 있는 경우
            if actions:
                # 최상의 행동 결정 (가장 높은 Q-값을 가진 행동)
                # 상태가 종료 상태이면 최적 행동을 'Terminal'로 표시
                best_action = max(actions, key=actions.get) if state not in terminal_states else 'Terminal'

                # 데이터 리스트에 상태, Q-값, 최적 행동 추가
                es_q_policy_data.append({
                    'State': state,
                    'up': actions.get('up', 0.0),  # 'up' 행동의 Q-값 (없으면 0.0)
                    'down': actions.get('down', 0.0),  # 'down' 행동의 Q-값
                    'left': actions.get('left', 0.0),  # 'left' 행동의 Q-값
                    'right': actions.get('right', 0.0),  # 'right' 행동의 Q-값
                    'Optimal Action': best_action  # 현재 상태의 최적 행동
                })
            else:
                # 상태에 유효한 Q-값이 없으면 기본값 추가
                es_q_policy_data.append({
                    'State': state,
                    'up': 0.0,
                    'down': 0.0,
                    'left': 0.0,
                    'right': 0.0,
                    'Optimal Action': 'N/A'  # 유효한 행동이 없음을 'N/A'로 표시
                })
        else:
            # 상태가 Q-테이블에 없으면 기본값 추가
            es_q_policy_data.append({
                'State': state,
                'up': 0.0,
                'down': 0.0,
                'left': 0.0,
                'right': 0.0,
                'Optimal Action': 'N/A'  # 누락된 상태를 'N/A'로 표시
            })

# 가독성을 위해 상태별로 데이터 정렬
es_q_policy_data.sort(key=lambda x: x['State'])

# 표 출력 헤더 정의
header = ['상태', '위', '아래', '왼쪽', '오른쪽', '최적 행동']

# 헤더 출력
print(f"{header[0]:<10} {header[1]:<10} {header[2]:<10} {header[3]:<10} {header[4]:<10} {header[5]:<15}")
print("-" * 65)  # 구분선 출력

# Q-값 및 최적 행동의 각 행 출력
for row_data in es_q_policy_data:
    print(f"{str(row_data['State']):<10} {row_data['up']:<10.2f} {row_data['down']:<10.2f} {row_data['left']:<10.2f} {row_data['right']:<10.2f} {row_data['Optimal Action']:<15}")

**표 형식 기대 SARSA Q-값 및 정책 분석**  

**주요 관찰 사항:**  

**1. Q-값 추세 및 가치 전파:**  
- **목표에 가까운 상태에서 더 높은 Q-값**이 관찰되며, 이는 에이전트가 장기 보상을 효과적으로 추정하는 법을 학습했음을 나타냅니다.  
- 예를 들어, **상태 (3,2)**는 종료 목표로 직접 이어지므로 오른쪽으로 이동하는 것에 대한 높은 Q-값을 가집니다.  
- **Q-값은 목표에서 외부로 전파**되며, 상태가 목표에서 멀어질수록 점진적으로 감소합니다.

**2. 최적 행동 선택 및 정책 구조:**  
- **최적 행동** 열은 에이전트가 목표를 향한 논리적인 경로를 따른다는 것을 확인시켜 줍니다.
- **오른쪽(→) 및 아래(↓) 행동**이 대부분의 상태에서 우세하며, 이는 목표로 가는 효율적인 경로와 일치합니다.
- **간혹 위(↑) 및 왼쪽(←) 선택**이 나타나는데, 이는 대안 경로가 탐색된 곳으로, **온-폴리시 학습**(SARSA의 엡실론-그리디 행동)의 영향을 반영합니다.

**3. 종료 상태 및 학습 행동:**  
- **목표 상태 (3,3)를 포함한 종료 상태**가 올바르게 식별되어 추가 행동 선택을 방지합니다.
- **종료 상태의 Q-값은 미래 보상에 기여하지 않으므로 0으로 유지**됩니다.

**4. 위험 인지 행동 및 절벽 회피:**  
- **위험 지역(절벽이나 장애물 등)에 인접한 상태**는 다양한 Q-값을 보여주며, 이는 에이전트가 다른 행동의 위험을 평가하는 법을 학습했음을 나타냅니다.
- SARSA의 **온-폴리시 학습** 접근 방식은 보상을 잃을 수 있는 위험한 상태를 피하는 **더 신중한 전략**을 낳습니다.
- 이러한 위험 회피 경향은 더 위험한 움직임으로 더 공격적인 정책을 학습했을 수 있는 **Q-러닝과의 주요 차이점**입니다.

**5. 히트맵 시각화와의 일치:**  
- **표 형식 Q-값은 히트맵 시각화와 일치**하며, 정책 구조가 일관되게 유지됨을 확인합니다.
- 행동 선택의 사소한 차이는 **여러 행동이 유사한 Q-값을 가질 때 동점 처리**에 기인할 수 있습니다.

**Q-러닝과의 비교:**
- **SARSA의 온-폴리시 업데이트**는 안전을 우선시하여 절벽 근처의 위험한 움직임을 피합니다.
- **Q-러닝(오프-폴리시)**은 오직 가장 높은 기대 수익에만 초점을 맞춰 **더 공격적인** 정책을 낳았을 수 있습니다.
- 이 차이는 특히 **절벽 인접 상태**에서 두드러지며, SARSA가 더 신중합니다.

# 다른 하이퍼파라미터로 테스트하기 (선택 사항)

학습률 (α), 할인 인자 (γ), 초기 엡실론 (ε₀)에 대해 다른 값을 실험하여 기대 SARSA의 학습 속도와 수렴에 미치는 영향을 관찰합니다.

In [ ]:
# --- 기대 SARSA 하이퍼파라미터 실험 실행 ---

# 실험을 위한 하이퍼파라미터 범위 정의
learning_rates_es_exp = [0.1, 0.5]  # 테스트할 다른 학습률 (alpha)
discount_factors_es_exp = [0.9, 0.99]  # 테스트할 다른 할인 인자 (gamma)
exploration_rates_es_exp = [1.0]  # 이 비교를 위한 고정 초기 엡실론

# 실험 결과를 저장할 리스트
es_results_exp = []

# 실험 시작을 알리는 메시지 출력
print("기대 SARSA 하이퍼파라미터 실험 실행 중...")

# 학습률, 할인 인자, 탐험률의 모든 조합에 대해 반복
for alpha in learning_rates_es_exp:
    for gamma in discount_factors_es_exp:
        for initial_epsilon in exploration_rates_es_exp:
            # 현재 테스트 중인 하이퍼파라미터 조합 출력
            print(f"  ES 훈련 중: alpha={alpha}, gamma={gamma}, epsilon_init={initial_epsilon}")
            
            # 현재 하이퍼파라미터 조합으로 기대 SARSA 실행
            q_table, rewards_per_episode, episode_lengths = run_expected_sarsa(
                state_space, action_space, rewards, terminal_states, rows, cols, alpha, gamma,
                initial_epsilon, min_epsilon_es, decay_rate_es, episodes_es, max_steps_es  # 이전과 동일한 최소/감쇠율 사용
            )
            
            # 이 조합에 대한 결과 저장
            es_results_exp.append({
                'alpha': alpha,  # 학습률
                'gamma': gamma,  # 할인 인자
                'initial_epsilon': initial_epsilon,  # 초기 탐험률
                'rewards_per_episode': rewards_per_episode,  # 에피소드별 보상
                'episode_lengths': episode_lengths  # 에피소드 길이
            })

# 실험 완료를 알리는 메시지 출력
print("실험 완료.")

# --- 시각화 ---

# 시각화에 필요한 서브플롯 수 결정
num_results_es = len(es_results_exp)  # 총 실험 수
plot_rows_es = int(np.ceil(np.sqrt(num_results_es)))  # 서브플롯 그리드의 행 수
plot_cols_es = int(np.ceil(num_results_es / plot_rows_es))  # 서브플롯 그리드의 열 수

# 모든 하이퍼파라미터 조합을 시각화하기 위한 더 큰 그림 생성
plt.figure(figsize=(20, 5))

# 결과를 반복하며 각 실험에 대한 보상 플로팅
for i, result in enumerate(es_results_exp):
    plt.subplot(plot_rows_es, plot_cols_es, i + 1)  # 현재 실험을 위한 서브플롯 생성
    plt.plot(result['rewards_per_episode'])  # 에피소드별 보상 플로팅
    plt.title(f"기대 SARSA: α={result['alpha']}, γ={result['gamma']}, ε₀={result['initial_epsilon']}")  # 하이퍼파라미터 포함 제목
    plt.xlabel('에피소드')  # x축 레이블
    plt.ylabel('총 보상')  # y축 레이블
    plt.grid(True)  # 가독성 향상을 위해 그리드 추가
    # 모든 실험에 걸친 최소 및 최대 보상을 기반으로 y축 한계 설정
    plt.ylim(
        min(min(r['rewards_per_episode']) for r in es_results_exp) - 1,
        max(max(r['rewards_per_episode']) for r in es_results_exp) + 1
    )


# 전체 그림에 대한 상위 제목 추가
plt.suptitle("다른 하이퍼파라미터를 사용한 기대 SARSA 성능", fontsize=16, y=1.02)

# 겹침 방지 및 플롯 표시를 위한 레이아웃 조정
plt.tight_layout()
plt.show()

# 다른 환경에 기대 SARSA 적용하기 (절벽 걷기)

이제 절벽 걷기 환경에 기대 SARSA를 적용하고 그 행동을 비교합니다. SARSA처럼 더 안전한 경로를 학습할 것으로 예상되며, 잠재적으로 더 부드러운 보상 개선이 있을 수 있습니다.

In [ ]:
# 비교를 위해 SARSA 절벽 실험의 하이퍼파라미터 재사용
alpha_cliff_es = 0.1
gamma_cliff_es = 0.99 # 더 높은 감마 사용
initial_epsilon_cliff_es = 0.2 # 잠재적으로 더 빠른 안전 경로 찾기를 위해 낮은 탐험으로 시작
min_epsilon_cliff_es = 0.01
decay_rate_cliff_es = 0.005
episodes_cliff_es = 500
max_steps_cliff_es = 200

# 환경 파라미터 다시 정의
cliff_rows, cliff_cols = 4, 12
cliff_start_state = (3, 0)
cliff_terminal_state = (3, 11)
cliff_states = [(3, c) for c in range(1, 11)]
cliff_action_space = ['up', 'down', 'left', 'right']

In [ ]:
# 보상 정의: 일반 스텝은 -1, 절벽은 -100, 목표는 +10 (암시적 목표 보상은 업데이트에서 처리됨)
# SARSA/Q-러닝은 일반적으로 상태로 *전환*하는 것에 대한 보상을 사용합니다.
# 표준 절벽 걷기 보상을 시뮬레이션할 수 있습니다: 스텝당 -1, 떨어지면 -100.
# 이 일반적인 설정을 위해 보상/전이 로직을 약간 수정해 봅시다.

# 표준 절벽 걷기를 위한 수정된 상태 전이 및 보상
def cliff_state_transition_reward(
    state: Tuple[int, int],
    action: str,
    rows: int,
    cols: int,
    cliff_states: List[Tuple[int, int]],
    start_state: Tuple[int, int]
) -> Tuple[Tuple[int, int], int]:
    """
    절벽 걷기의 다음 상태와 보상을 계산합니다.
    일반 스텝은 보상 -1, 절벽으로 떨어지면 -100.
    절벽으로 떨어지면 상태를 start_state로 재설정합니다.
    """
    row, col = state
    next_row, next_col = row, col

    # 잠재적 다음 위치 계산
    if action == 'up' and row > 0:
        next_row -= 1
    elif action == 'down' and row < rows - 1:
        next_row += 1
    elif action == 'left' and col > 0:
        next_col -= 1
    elif action == 'right' and col < cols - 1:
        next_col += 1

    next_state = (next_row, next_col)

    # 보상 결정
    if next_state in cliff_states:
        reward = -100
        next_state = start_state # 절벽에 빠지면 시작 상태로 리셋
    elif next_state == cliff_terminal_state:
        reward = 0 # 표준 설정은 종종 목표 도달 시 0을 제공, -1 전환 비용은 스텝 보상으로 처리됨
                   # 대안: 원할 경우 여기에 +10 또는 다른 양의 보상을 제공. -1 스텝 비용을 고수하자.
        reward = -1 # 일관성을 위해 목표 도달에도 스텝 비용 적용? 아니, 목표 전환은 특별함.
                    # 일반적인 -1 스텝 비용 구조를 사용하자. 목표 상태 자체는 도착 시 보상/페널티 없음.
        reward = -1 # 주석대로 -1로 설정
    else:
        reward = -1 # 표준 스텝 비용

    return next_state, reward

In [ ]:
# 절벽 걷기 환경을 위한 특정 기대 SARSA 실행기 정의
def run_expected_sarsa_cliff_episode(
    q_table: Dict[Tuple[int, int], Dict[str, float]],  # 상태를 행동 및 Q-값에 매핑하는 Q-테이블
    action_space: List[str],  # 가능한 행동 리스트
    terminal_state: Tuple[int, int],  # 환경의 종료 상태
    cliff_states: List[Tuple[int, int]],  # 환경의 절벽 상태 리스트
    start_state: Tuple[int, int],  # 에피소드의 시작 상태
    rows: int,  # 그리드의 행 수
    cols: int,  # 그리드의 열 수
    alpha: float,  # 학습률 (0 < alpha <= 1)
    gamma: float,  # 할인 인자 (0 <= gamma <= 1)
    epsilon: float,  # 탐험률 (0 <= epsilon <= 1)
    max_steps: int  # 에피소드당 최대 스텝 수
) -> Tuple[int, int]:
    """
    절벽 걷기 환경에 대해 기대 SARSA의 단일 에피소드를 실행합니다.

    Parameters:
    - q_table: 모든 상태-행동 쌍에 대한 Q-값을 저장하는 Q-테이블.
    - action_space: 모든 가능한 행동 리스트.
    - terminal_state: 환경의 종료 상태.
    - cliff_states: 환경의 절벽 상태 리스트.
    - start_state: 에피소드의 시작 상태.
    - rows: 그리드의 행 수.
    - cols: 그리드의 열 수.
    - alpha: 학습률 (0 < alpha <= 1).
    - gamma: 할인 인자 (0 <= gamma <= 1).
    - epsilon: 탐험률 (0 <= epsilon <= 1).
    - max_steps: 에피소드당 허용되는 최대 스텝 수.

    Returns:
    - Tuple containing:
        - total_reward: 에피소드 동안 누적된 총 보상.
        - steps: 에피소드에서 취한 스텝 수.
    """
    # 시작 상태 초기화
    state: Tuple[int, int] = start_state
    total_reward: int = 0  # 에피소드의 총 보상 누적
    steps: int = 0  # 에피소드에서 취한 스텝 수 추적

    # 최대 스텝 수만큼 반복
    for _ in range(max_steps):
        # 엡실론-그리디 정책을 사용하여 행동 선택
        action: str = epsilon_greedy_policy(state, q_table, action_space, epsilon)

        # 선택된 행동을 취하고 다음 상태와 보상 관찰
        next_state, reward = cliff_state_transition_reward(
            state, action, rows, cols, cliff_states, start_state
        )
        total_reward += reward  # 총 보상 업데이트

        # 기대 SARSA 규칙을 사용하여 현재 상태-행동 쌍에 대한 Q-값 업데이트
        # 업데이트 함수에 종료 상태를 리스트로 전달
        update_expected_sarsa_value(
            q_table, state, action, reward, next_state, alpha, gamma, epsilon, action_space, [terminal_state]
        )

        # 다음 상태로 이동
        state = next_state
        steps += 1  # 스텝 카운터 증가

        # 종료 상태에 도달하면 에피소드 종료
        if state == terminal_state:
            break

    return total_reward, steps

In [ ]:
def run_expected_sarsa_cliff(
    action_space: List[str],  # 가능한 행동 리스트 (예: ['up', 'down', 'left', 'right'])
    terminal_state: Tuple[int, int],  # 환경의 종료 상태
    cliff_states: List[Tuple[int, int]],  # 환경의 절벽 상태 리스트
    start_state: Tuple[int, int],  # 각 에피소드의 시작 상태
    rows: int,  # 그리드의 행 수
    cols: int,  # 그리드의 열 수
    alpha: float,  # 학습률 (0 < alpha <= 1)
    gamma: float,  # 할인 인자 (0 <= gamma <= 1)
    initial_epsilon: float,  # 초기 탐험률 (0 <= epsilon <= 1)
    min_epsilon: float,  # 최소 탐험률
    decay_rate: float,  # 엡실론 감쇠율
    episodes: int,  # 훈련할 에피소드 수
    max_steps: int  # 에피소드당 최대 스텝 수
) -> Tuple[Dict[Tuple[int, int], Dict[str, float]], List[int], List[int]]:
    """
    절벽 걷기 환경에 대해 기대 SARSA 알고리즘을 실행합니다.

    Parameters:
    - action_space (List[str]): 모든 가능한 행동 리스트.
    - terminal_state (Tuple[int, int]): 환경의 종료 상태.
    - cliff_states (List[Tuple[int, int]]): 환경의 절벽 상태 리스트.
    - start_state (Tuple[int, int]): 각 에피소드의 시작 상태.
    - rows (int): 그리드의 행 수.
    - cols (int): 그리드의 열 수.
    - alpha (float): 학습률 (0 < alpha <= 1).
    - gamma (float): 할인 인자 (0 <= gamma <= 1).
    - initial_epsilon (float): 초기 탐험률 (0 <= epsilon <= 1).
    - min_epsilon (float): 최소 탐험률.
    - decay_rate (float): 엡실론 감쇠율.
    - episodes (int): 훈련할 에피소드 수.
    - max_steps (int): 에피소드당 최대 스텝 수.

    Returns:
    - Tuple containing:
        - q_table (Dict[Tuple[int, int], Dict[str, float]]): 모든 상태-행동 쌍에 대한 Q-값을 저장하는 Q-테이블.
        - rewards_per_episode (List[int]): 각 에피소드에서 누적된 총 보상 리스트.
        - episode_lengths (List[int]): 각 에피소드에서 취한 스텝 수 리스트.
    """
    # 상태 공간을 모든 가능한 (행, 열) 쌍으로 생성
    state_space: List[Tuple[int, int]] = [(r, c) for r in range(rows) for c in range(cols)]

    # 모든 상태-행동 쌍에 대해 Q-테이블을 0으로 초기화
    q_table: Dict[Tuple[int, int], Dict[str, float]] = initialize_q_table_nested(state_space, action_space)

    # 각 에피소드의 보상 및 에피소드 길이를 저장할 리스트
    rewards_per_episode: List[int] = []
    episode_lengths: List[int] = []

    # 각 에피소드 루프
    for episode in range(episodes):
        # 현재 에피소드에 대한 엡실론 동적 조정
        epsilon: float = adjust_epsilon(initial_epsilon, min_epsilon, decay_rate, episode)

        # 절벽 걷기 환경에 대한 기대 SARSA의 단일 에피소드 실행
        total_reward, steps = run_expected_sarsa_cliff_episode(
            q_table, action_space, terminal_state, cliff_states, start_state,
            rows, cols, alpha, gamma, epsilon, max_steps
        )

        # 총 보상 및 에피소드 길이 저장
        rewards_per_episode.append(total_reward)
        episode_lengths.append(steps)

    # Q-테이블, 에피소드별 보상, 에피소드 길이 반환
    return q_table, rewards_per_episode, episode_lengths


절벽 걷기에 대해 기대 SARSA를 실행해 봅시다.

In [ ]:
# 비교를 위해 SARSA 절벽 실험의 하이퍼파라미터 재사용
alpha_cliff_es = 0.1  # 학습률: Q-값 업데이트의 스텝 크기 제어
gamma_cliff_es = 0.99  # 할인 인자: 미래 보상의 중요도 결정
initial_epsilon_cliff_es = 0.2  # 초기 탐험률: 무작위 행동 선택 확률
min_epsilon_cliff_es = 0.01  # 최소 탐험률: 엡실론의 하한값
decay_rate_cliff_es = 0.005  # 엡실론 감쇠율: 엡실론 감소 속도 제어
episodes_cliff_es = 500  # 에이전트 훈련 에피소드 수
max_steps_cliff_es = 200  # 에피소드당 허용되는 최대 스텝 수

# 훈련 시작을 알리는 메시지 출력
print("절벽 걷기 환경에서 기대 SARSA 실행 중...")

# 절벽 걷기 환경 파라미터 정의
cliff_rows, cliff_cols = 4, 12  # 그리드 차원 (4 행 x 12 열)
cliff_start_state = (3, 0)  # 각 에피소드의 시작 상태
cliff_terminal_state = (3, 11)  # 종료 상태 (목표 상태)
cliff_states = [(3, c) for c in range(1, 11)]  # 절벽 상태 (피해야 할 위험한 상태)
cliff_action_space = ['up', 'down', 'left', 'right']  # 환경에서 가능한 행동

# 절벽 걷기 환경에서 기대 SARSA 훈련 실행
cliff_q_table_es, cliff_rewards_es, cliff_lengths_es = run_expected_sarsa_cliff(
    cliff_action_space,  # 가능한 행동 리스트
    cliff_terminal_state,  # 종료 상태
    cliff_states,  # 절벽 상태 리스트
    cliff_start_state,  # 각 에피소드의 시작 상태
    cliff_rows,  # 그리드의 행 수
    cliff_cols,  # 그리드의 열 수
    alpha_cliff_es,  # 학습률
    gamma_cliff_es,  # 할인 인자
    initial_epsilon_cliff_es,  # 초기 탐험률
    min_epsilon_cliff_es,  # 최소 탐험률
    decay_rate_cliff_es,  # 엡실론 감쇠율
    episodes_cliff_es,  # 훈련할 에피소드 수
    max_steps_cliff_es  # 에피소드당 최대 스텝 수
)

# 훈련 완료를 알리는 메시지 출력
print("절벽 걷기에서 기대 SARSA 훈련 완료.")

보상을 플로팅하기 위해 다음 코드를 사용할 수 있습니다.

In [ ]:
# 절벽 걷기 환경에 대한 보상 플로팅
def plot_rewards(rewards_per_episode: List[int], ax: plt.Axes = None) -> plt.Axes:
    """
    에피소드에 걸쳐 누적된 총 보상을 플로팅합니다.

    Parameters:
    - rewards_per_episode (List[int]): 에피소드별 총 보상 리스트.
    - ax (plt.Axes, optional): 플로팅할 Matplotlib 축. None이면 새 그림과 축이 생성됩니다.

    Returns:
    - plt.Axes: 플롯을 포함하는 Matplotlib 축.
    """
    # 축이 제공되지 않으면 새 그림과 축 생성
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))
    
    # 에피소드에 걸쳐 보상 플로팅
    ax.plot(rewards_per_episode)
    ax.set_xlabel('에피소드')  # x축 레이블
    ax.set_ylabel('총 보상')  # y축 레이블
    ax.set_title('에피소드별 보상')  # 플롯 제목
    
    # 필요한 경우 추가 사용자 정의를 위해 축 반환
    return ax

결과를 시각화하기 위해 이전과 유사하게 보상과 에피소드 길이를 플로팅할 수 있습니다. 또한 절벽 걷기 환경에 대해 학습된 정책을 시각화해 봅시다.

In [ ]:
# --- 절벽 걷기 시각화 (기대 SARSA) ---
fig_cliff_es, axs_cliff_es = plt.subplots(2, 2, figsize=(18, 8))

# 보상
plot_rewards(cliff_rewards_es, ax=axs_cliff_es[0, 0])
axs_cliff_es[0, 0].set_title("기대 SARSA: 절벽 걷기 보상")
axs_cliff_es[0, 0].grid(True)

# 에피소드 길이
axs_cliff_es[0, 1].plot(cliff_lengths_es)
axs_cliff_es[0, 1].set_xlabel('에피소드')
axs_cliff_es[0, 1].set_ylabel('에피소드 길이')
axs_cliff_es[0, 1].set_title('기대 SARSA: 절벽 걷기 에피소드 길이')
axs_cliff_es[0, 1].grid(True)

# 최대 Q-값 플로팅 (히트맵) - 절벽 환경 파라미터를 받는 함수 필요
def plot_q_values_cliff(q_table, rows, cols, ax):
    q_values = np.zeros((rows, cols))
    for r in range(rows):
        for c in range(cols):
            state = (r, c)
            if state in q_table and q_table[state]:
                q_values[r, c] = max(q_table[state].values())
            else:
                q_values[r,c] = -np.inf # 방문하지 않았거나 종료 상태 표시

    # 더 나은 시각화를 위해 절벽 상태 마스크
    q_values_masked = np.ma.masked_where(q_values <= -100, q_values) # 절벽의 극단적인 음수 값 숨기기

    im = ax.imshow(q_values_masked, cmap='viridis')
    plt.colorbar(im, ax=ax, label='최대 Q-값')
    ax.set_title('기대 SARSA: 최대 Q-값 (절벽)')
    ax.set_xticks(np.arange(cols))
    ax.set_yticks(np.arange(rows))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(which='major', color='w', linestyle='-', linewidth=1)

# 최대 Q-값
plot_q_values_cliff(cliff_q_table_es, cliff_rows, cliff_cols, ax=axs_cliff_es[1, 0])
axs_cliff_es[1, 0].set_title('기대 SARSA: 최대 Q-값 (절벽)')

# 정책
plot_policy_grid(cliff_q_table_es, cliff_rows, cliff_cols, [cliff_terminal_state], ax=axs_cliff_es[1, 1])
axs_cliff_es[1, 1].set_title("기대 SARSA: 학습된 정책 (절벽)")

plt.tight_layout()
plt.show()

**절벽 걷기에 대한 기대 SARSA 업데이트된 분석:**

**1. 보상 및 에피소드 길이 (상단 행):**
- **총 보상 그래프**는 초기에 급격한 하락을 보여주며, 이는 에이전트가 절벽으로 자주 떨어져 큰 음수 보상을 받는 초기 에피소드를 반영합니다.
- 시간이 지남에 따라 에이전트가 절벽을 피하고 더 안전한 경로를 따르는 법을 배우면서 **보상이 개선되고 안정화**됩니다.
- **에피소드 길이는 일관되게 감소**하여 에이전트가 더 적은 스텝으로 목표에 더 효율적으로 도달하는 법을 배우고 있음을 나타냅니다.
- **더 부드러운 수렴 패턴**은 기대 SARSA의 기대값 업데이트가 표준 SARSA에 비해 더 안정적인 학습을 제공함을 시사합니다.

**2. 최대 Q-값 및 정책 (하단 행):**
- **Q-값 히트맵**은 학습 과정을 강화하며, 절벽 근처의 낮은 값(어두운 영역)은 떨어지는 것의 높은 페널티를 강조합니다.
- **더 높은 Q-값(밝은 영역)은 더 안전한 경로를 따라 집중**되어 에이전트의 선호 경로를 나타냅니다.
- **학습된 정책 그리드**는 에이전트가 즉각적인 위험을 피하기 위해 초기에 **위쪽**으로 이동한 다음, **상단 행을 따라 오른쪽**으로 진행하고 **목표로 아래쪽**으로 이동하는 전략을 보여줍니다.
- 이 보수적인 경로는 에이전트가 절벽 영역으로 들어갈 위험을 최소화하도록 보장하며, **기대 SARSA의 위험 인지적 성격**을 강조합니다.

**절벽 걷기에 대한 결론 (기대 SARSA):**
기대 SARSA는 더 위험하지만 짧은 경로보다 안전한 경로를 선호하는 **안정적이고 신중한 정책을 성공적으로 학습**합니다.  
표준 SARSA와 비교하여:
- 기대값 업데이트로 인해 변동성을 줄여 **더 부드럽게 수렴**합니다.
- 최종 정책은 **온-폴리시이며 위험 회피적**으로 남아 절벽을 피하면서 효율성을 유지합니다.
- **기대 SARSA는 안정성과 위험 최소화가 중요한 환경에서 효과적임**을 증명합니다.

## 일반적인 문제점과 해결책

(SARSA에서 재사용, 문제점은 유사함)

**문제점: 느린 학습 또는 막힘**
*   **해결책**: 하이퍼파라미터(α, γ, ε 감쇠) 조정. 초기에 탐험을 늘리거나 더 정교한 탐험 사용. 더 큰 상태 공간에는 함수 근사 사용.

**문제점: 탐험과 활용의 균형 맞추기**
*   **해결책**: 잘 조정된 엡실론 감쇠 스케줄 사용.

**문제점: 적절한 하이퍼파라미터 선택**
*   **해결책**: 실험. 일반적인 값(α≈0.1, γ≈0.9-0.99, ε는 1.0에서 0.1/0.01로 감쇠)으로 시작.

**문제점: 기대값 계산 비용**
*   **해결책**: 행동 수가 적은 테이블형의 경우 합계는 저렴합니다. 크거나 연속적인 행동 공간의 경우 정확한 기대값 계산은 불가능하며, 다른 기술(예: 샘플링, 정책에 대한 함수 근사)이 필요합니다.

## 기대 SARSA vs. 다른 강화 학습 알고리즘

### 기대 SARSA의 장점
-   **온-폴리시(On-Policy):** 따르고 있는 정책의 가치를 학습합니다.
-   **낮은 분산:** 단일 샘플에 의존하는 대신 가능한 다음 행동에 대해 평균을 내기 때문에 일반적으로 SARSA보다 업데이트 분산이 낮습니다. 이는 더 안정적이고 때로는 더 빠른 학습으로 이어질 수 있습니다.
-   **종종 더 안정적/보수적:** 위험한 환경에서 더 안전한 정책을 배우는 SARSA의 경향을 계승합니다.
-   표준 조건 하에서 수렴이 보장됩니다.

### 기대 SARSA의 한계점
-   **Q-러닝보다 느릴 수 있음:** 여전히 현재 정책을 기반으로 학습하며, 이는 차선의 행동을 탐험할 수 있어 오프-폴리시 Q-러닝에 비해 진정한 최적 정책으로의 수렴을 잠재적으로 늦출 수 있습니다.
-   **계산 비용:** 다음 상태의 모든 행동을 반복하여 기대값을 계산해야 하므로, 행동 공간이 큰 경우 SARSA나 Q-러닝보다 업데이트당 계산 비용이 약간 더 많이 들 수 있습니다 (작고 이산적인 행동 공간에서는 무시할 수 있음). 수정 없이는 연속적인 행동 공간에 직접 적용할 수 없습니다.
-   **상태 공간 한계:** 테이블 버전은 큰 상태 공간에서 어려움을 겪습니다.

### 관련 알고리즘
-   **SARSA**: 기대 SARSA가 파생된 기본 온-폴리시 TD 알고리즘입니다.
-   **Q-러닝**: 오프-폴리시 TD 알고리즘으로, 최적 정책을 직접 학습합니다.
-   **SARSA(λ) / 기대 SARSA(λ)**: 더 빠른 신용 할당을 위해 자격 추적(eligibility traces)을 사용하는 버전입니다.
-   **액터-크리틱 방법**: 정책과 가치 함수를 모두 학습합니다.

## 결론

기대 SARSA는 업데이트 분산을 줄임으로써 표준 SARSA를 개선하는 효과적인 온-폴리시 시간차 제어 알고리즘입니다. 이는 샘플링된 단일 다음 행동에 의존하는 대신, 현재 정책에 따라 모든 행동에 대해 평균을 낸 다음 상태의 *기대* Q-값을 사용하여 이를 달성합니다.

이는 종종 더 부드럽고 잠재적으로 더 빠른 수렴으로 이어지면서 SARSA의 온-폴리시 특성을 유지하므로, 현재 정책을 평가하고 절벽 걷기와 같은 위험한 환경에서 더 안전한 행동을 학습하는 데 적합합니다. 테이블형의 경우 단계당 계산 비용이 SARSA보다 약간 더 많이 들지만, 향상된 안정성 덕분에 강력한 대안 온-폴리시 학습 방법이 됩니다.